In [ ]:
import os,sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
from PIL import Image
from sklearn.model_selection import train_test_split

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
import scipy
from scipy import ndimage
import tensorflow as tf

In [ ]:
df_label = pd.read_csv("cxr_label_train.csv")
Filter = (df_label["PATIENT ID"] == int(638))
df_label[Filter].iloc[0,1]

In [ ]:
train_path = './IML_CXR/'
# train_path = '/content/drive/MyDrive/Colab Notebooks/IML_CXR/'
test_path = './IML_CXR_TEST/'
# test_path = '/content/drive/MyDrive/Colab Notebooks/IML_CXR_TEST/'
count = 0
img_train = os.listdir(train_path)
img_test = os.listdir(test_path)
X = np.empty((len(img_train), 224, 224, 3))
Y = np.empty((len(img_train), 2))

for img_name in img_train:
    img_path = train_path + img_name
    img = image.load_img(img_path, target_size=(224, 224))
    img = image.img_to_array(img) / 255.0
    PatientID = img_name.split('.jpg')
    condition = (df_label["PATIENT ID"] == int(PatientID[0]))
    label = df_label[condition].iloc[0,1] #take image label
    X[count] = img
    if label == 0:
        Y[count] = np.array((1,0))
    else :
        Y[count] = np.array((0,1))
    count += 1


In [ ]:
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,random_state = 777,test_size = 0.2)

In [ ]:
model = ResNet50(
    weights=None,
    classes=2
)

In [ ]:
model.compile(optimizer=tf.keras.optimizers.Adam(),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
model.fit(X_train, Y_train, epochs=2, batch_size=8)

In [ ]:
img_path = "./IML_CXR/638.jpg"
img = image.load_img(img_path, target_size=(224, 224))

plt.imshow(img)
img = image.img_to_array(img) / 255.0
img = np.expand_dims(img, axis=0)  

print(model.predict(img))
np.argmax(model.predict(img))

In [ ]:
from sklearn import metrics
predictions = model.predict(X_test)
np.set_printoptions(threshold=np.inf)

In [ ]:
print(predictions)
X_ = []
Y_ = []
for x in predictions:
    X_.append(np.argmax(x))
for y in Y_test:
    Y_.append(np.argmax(y))
print(X)

In [ ]:
print('Precision:', metrics.precision_score(Y_test, predictions))
print('Recall:', metrics.recall_score(Y_test, predictions))

In [ ]:
model.evaluate(X_test, Y_test, batch_size=32)